In [12]:
%%capture
!pip install yfinance

In [13]:
import numpy as np
import pandas as pd
import datetime

import yfinance as yf

import cvxopt as opt
from cvxopt import solvers, matrix

pd.options.display.float_format = '{:.6f}'.format

In [23]:
class Crypto:
    def __init__(self, tickers, start_date, end_date):
        self.tickers = tickers
        self.start_date = start_date
        self.end_date = end_date
        self.close_prices = self.initialize_close_prices()

    def initialize_close_prices(self):
        close_prices = pd.DataFrame()

        # Get close prices for each ticker
        for ticker in self.tickers:
            crypto = yf.Ticker(ticker)
            close_price = crypto.history(start=self.start_date, end=self.end_date)['Close']
            close_prices[ticker] = close_price

        # Drop rows that contain missing values
        close_prices.dropna(inplace=True)

        # Convert the index to datetime objects with the desired format
        close_prices.index = pd.to_datetime(close_prices.index, format='%Y-%m-%d')

        # Extract the year-month-day from the datetime objects and use it as the new index
        close_prices.index = close_prices.index.strftime('%Y-%m-%d')

        return close_prices

    def get_close_prices(self):
        return self.close_prices

    def minimum_variance_portfolio_with_positive_weights(self, target_return=None):
        # Calculate daily returns
        returns = self.close_prices.pct_change().dropna()

        # Check if we can achieve desiable target_return
        mean_ret = returns.mean()
        min_ret, max_ret = mean_ret.min(), mean_ret.max()
        if target_return is not None and (target_return < min_ret or target_return > max_ret):
            raise ValueError(f'Inappropriate value of target return. It should be between {min_ret} and {max_ret}')

        # Calculate covariance matrix
        cov_matrix = returns.cov()
        
        # Convert covariance matrix to cvxopt matrix
        Sigma = opt.matrix(cov_matrix.values)

        # Create constraint matrices
        n = len(returns.columns)
        pbar = opt.matrix(np.ones(n))
        G = opt.matrix(np.vstack((-np.eye(n), np.eye(n))))  # negative and positive n x n identity matrices; Gx <= h
        h = opt.matrix(np.vstack((np.zeros((n, 1)), np.ones((n, 1)))))  # n x 1 matrices of zeros and ones
        A = opt.matrix(1.0, (1, n))  # Ax = b
        b = opt.matrix(1.0)

        if target_return is not None:
            # Add target return constraint: mu'x >= target_return
            mu = opt.matrix(mean_ret)
            G = opt.matrix(np.vstack((G, -mu.T)))
            h = opt.matrix(np.vstack((h, -target_return)))
    
        # Calculate efficient frontier weights using quadratic programming
        solvers.options['show_progress'] = False
        portfolio_weights = solvers.qp(Sigma, pbar, G, h, A, b)  #['x']

        # Check feasibility
        if portfolio_weights['status'] == 'optimal':
            portfolio_weights = np.array(portfolio_weights['x']).flatten().tolist()
            weight_map = pd.DataFrame({'Crypto': self.close_prices.columns, 
                                       'Daily_Return': mean_ret.values,
                                       'Std': np.sqrt(np.diag(cov_matrix)),
                                       'Weight': [float(i) for i in portfolio_weights]})
            return weight_map
        else:
            raise ValueError(f'Feasible solution not found for target return of {target_return}')


def get_portfolio_variance(w, cov_matrix):
    return np.dot(np.dot(w.T, cov_matrix), w)

def get_correlation_matrix(cov_matrix):
    # Calculate the standard deviations for each variable
    std_devs = np.sqrt(np.diag(cov_matrix))

    # Divide the covariance matrix by the product of the standard deviations
    correlation_matrix = cov_matrix / np.outer(std_devs, std_devs)

    # Replace any NaN or infinite values with 0
    correlation_matrix = np.nan_to_num(correlation_matrix, 0)

    # Convert the numpy array to a pandas dataframe and set the column and row names
    correlation_matrix = pd.DataFrame(correlation_matrix, index=cov_matrix.index, columns=cov_matrix.columns)

    return correlation_matrix

In [27]:
# Cryptocurrencies you want to include in the portfolio
# tickers = ['BTC-USD', 'ETH-USD', 'LTC-USD', 'ADA-USD', 'DASH-USD', 'BNB-USD', 'BCH-USD', 'XLM-USD', 'XRP-USD', 'TRX-USD']
tickers = ['BTC-USD', 'BNB-USD', 'XRP-USD', 'TRX-USD']

# Get the current date
now = datetime.datetime.now()

# Subtract 3 months from the current date
delta = datetime.timedelta(days=91)
start_date = now - delta

# Format the date in the required format
start_date = start_date.strftime("%Y-%m-%d")
end_date = now.strftime("%Y-%m-%d")

# Initialize Crypto class
Cryptos = Crypto(tickers, start_date, end_date)

# Get covariance matrix
close_prices = Cryptos.get_close_prices()
returns = close_prices.pct_change().dropna()
cov_matrix = returns.cov()

In [36]:
# Get minimum variance portfolio
min_var_weights = Cryptos.minimum_variance_portfolio_with_positive_weights()

# Get weights for each asset
w = min_var_weights['Weight']

# Get portfolio variance
p_var = get_portfolio_variance(w, cov_matrix)

# Get correlation matrix
cor_matrix = get_correlation_matrix(cov_matrix)

print(f'Estimated standard deviation of the portfolio is {np.sqrt(p_var)}')
print('-------------------------------------------')
print('Weights DataFrame:')
print('-------------------------------------------')
print(min_var_weights)
print('-------------------------------------------')
print('Correlation matrix:')
print('-------------------------------------------')
print(cor_matrix)

Estimated standard deviation of the portfolio is 0.020654943469069476
-------------------------------------------
Weights DataFrame:
-------------------------------------------
    Crypto  Daily_Return      Std   Weight
0  BTC-USD      0.003775 0.022286 0.337375
1  BNB-USD      0.000584 0.026215 0.081562
2  XRP-USD     -0.000234 0.024770 0.163361
3  TRX-USD      0.002992 0.022332 0.417702
-------------------------------------------
Correlation matrix:
-------------------------------------------
         BTC-USD  BNB-USD  XRP-USD  TRX-USD
BTC-USD 1.000000 0.713474 0.743279 0.759776
BNB-USD 0.713474 1.000000 0.674864 0.651698
XRP-USD 0.743279 0.674864 1.000000 0.658573
TRX-USD 0.759776 0.651698 0.658573 1.000000


In [ ]:
# Cryptos.minimum_variance_portfolio_with_positive_weights(target_return=0.003)